In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import control
import ipywidgets as widgets
import sympy as sp

# Definiamo s come simbolo di Laplace
s = sp.symbols('s')

# Funzione per convertire un'espressione simbolica in TransferFunction
def sympy_to_tf(expr):
    # Otteniamo numeratore e denominatore come polinomi
    num, den = sp.fraction(sp.simplify(expr))
    # Convertiamo in liste di coefficienti (dal termine più alto)
    num_coeffs = [float(coef) for coef in sp.Poly(num, s).all_coeffs()]
    den_coeffs = [float(coef) for coef in sp.Poly(den, s).all_coeffs()]
    return control.TransferFunction(num_coeffs, den_coeffs)

# Funzione per plottare la risposta
def plot_response(expr_input, input_type="step", freq=1.0, t_end=10):
    # Convertiamo l'input in espressione sympy
    try:
        expr = sp.sympify(expr_input)
    except Exception as e:
        print("Errore nell'espressione:", e)
        return
    
    # Convertiamo in TransferFunction
    try:
        system = sympy_to_tf(expr)
    except Exception as e:
        print("Errore nella conversione in TransferFunction:", e)
        return

    # Tempo di simulazione
    t = np.linspace(0, t_end, 1000)

    # Selezione input
    if input_type == "step":
        t, y = control.step_response(system, T=t)
        u = None
    elif input_type == "impulse":
        t, y = control.impulse_response(system, T=t)
        u = None
    elif input_type == "ramp":
        u = t
        t, y = control.forced_response(system, T=t, U=u)
    elif input_type == "sine":
        u = np.sin(freq * t)
        t, y = control.forced_response(system, T=t, U=u)
    else:
        raise ValueError("Input non supportato")

    # Plot
    plt.figure(figsize=(8,4))
    plt.plot(t, y, label="Output y(t)", color="tab:blue", linewidth=2)
    if u is not None:
        plt.plot(t, u, "--", label="Input u(t)", color="tab:orange", linewidth=1.5)
    plt.title(f"Time Response ({input_type})", fontsize=14)
    plt.xlabel("Tempo [s]", fontsize=12)
    plt.ylabel("Ampiezza", fontsize=12)
    plt.grid(True, linestyle="--", alpha=0.7)
    plt.legend()
    plt.show()

# Widget interattivi
expr_text = widgets.Text(value="(s+1)/(s+2)", description="TF (s):", layout=widgets.Layout(width='400px'))

input_buttons = widgets.ToggleButtons(
    options=["step", "impulse", "ramp", "sine"],
    value="step",
    description="Input:",
    button_style="info",
    tooltips=["Step", "Impulse", "Ramp", "Sine"],
    style={'description_width': 'initial'}
)

freq_slider = widgets.FloatSlider(value=1.0, min=0.1, max=10.0, step=0.1, description="Freq (sine):")
tend_slider = widgets.FloatSlider(value=10, min=1, max=50, step=1, description="Tempo max:")

ui = widgets.VBox([expr_text, input_buttons, freq_slider, tend_slider])

out = widgets.interactive_output(
    lambda expr_input, input_type, freq, t_end: plot_response(
        expr_input, input_type, freq, t_end
    ),
    {
        "expr_input": expr_text,
        "input_type": input_buttons,
        "freq": freq_slider,
        "t_end": tend_slider,
    }
)

display(ui, out)





Output()